Import packages needed

In [1]:
import suite2p
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from natsort import natsorted
from multiprocessing import Pool
import gc
gc.collect()

3711

Pre-set parameters, and load the only bin files in the pre-set path folder

In [3]:
opsSetfile=r'E:\LuRecordingSuit2pPro\ops3Planes.npy';
ops0 = np.load(opsSetfile,allow_pickle=True).item()
ops0['input_format']='binary';
#ops0['save_path0']='C:/Users/zhangl33/Projects/Project4-SLMonline/LuSLMOnlineTest/02082024/'
ops0['save_path0']=r'E:\LuRecording\03142024_MouseMei03_VGlutAi203SLM\\'
ops0['Lx']=512  ##Width of a image
ops0['Ly']=512  ##Height of a image
ops0['xrange']=[0,512]
ops0['yrange']=[0,512]
ops0['nplanes']=3
ops0['nchannels']=1
ops0['tau']=0.9
ops0['fs']=6.9;
#ops0['nbinned']=2500
nplanes=ops0['nplanes'];
binFile = glob.glob(ops0['save_path0'] + '/*-003.bin')
print(binFile)


## creat new folder to process the data

SaveFolder=os.path.join(ops0['save_path0'], 'suite2p');
print(SaveFolder)
if not os.path.exists(SaveFolder):
   os.makedirs(SaveFolder)
else:
   print("Folder already exists.")

['E:\\LuRecording\\03142024_MouseMei03_VGlutAi203SLM\\TSeries-03142024-0925-003.bin']
E:\LuRecording\03142024_MouseMei03_VGlutAi203SLM\\suite2p


Load bin files

In [4]:

os.path.exists(binFile[0])
if len(binFile)>1:
   print('More than one bin files found!!')
else:
   rawBin = suite2p.io.BinaryFile(Ly=ops0['Ly'],Lx=ops0['Lx'], filename=binFile[0])
   nFrame = rawBin.shape[0] // nplanes

#Make sure each plane has same number of frames.
FramePerPlane=np.floor(rawBin.shape[0]/ops0['nplanes'])
TotalFrameNeed=np.int32(np.floor(FramePerPlane)*ops0['nplanes'])
print(TotalFrameNeed)

3600


Suite2p processing with no motion correction; Noted following defining a function to process a plane of data; Next session, parallel processing for each plane is excuted. Much time is saved. 
Alternatively, run this process with Step2-OnlineBinSuite2p.ipynb

In [6]:
print(ops0['nbinned'])

5000


In [5]:
from joblib import Parallel, delayed
ops1=ops0

#ops1['save_path']='C:/Users/zhangl33/Projects/Project4-SLMonline/LuSLMOnlineTest/02082024/CommandLineProcess/'
def process_plane(plane_idx):
    print('Processing plane ' + str(plane_idx))
   # fpath = ops1["save_path"]
    fpath=os.path.join(SaveFolder, f'plane{plane_idx}/');
    if not os.path.exists(fpath):
       os.makedirs(fpath)
    else:
       print("Saving folder already exists.")
    
    #plane_data = rawBin[range(0+plane_idx,TotalFrameNeed,nplanes),:,::-1]
    plane_data = rawBin[range(0+plane_idx,TotalFrameNeed,nplanes),:,:]
    ops, stat = suite2p.detection_wrapper(f_reg=plane_data, ops=ops1)
    ops['nframes']=nFrame
    ops['meanImg']=np.mean(plane_data,axis=0)
    ops=suite2p.registration.register.enhanced_mean_image(ops)
    np.save(os.path.join(fpath, "ops.npy"), ops)
    stat_after_extraction, F, Fneu, F_chan2, Fneu_chan2 = suite2p.extraction_wrapper(stat, f_reg=plane_data,
                                                                   f_reg_chan2 = plane_data,ops=ops1)
#    np.save(os.path.join(fpath, "ops.npy"), ops)
    if len(stat) > 0:
       classfile=suite2p.classification.builtin_classfile
       iscell = suite2p.classify(stat=stat_after_extraction, classfile=classfile)

# Apply preprocessing step for deconvolution
       dF = F.copy() - ops['neucoeff']*Fneu

       dF = suite2p.extraction.preprocess(
            F=dF,
            baseline=ops1['baseline'],
            win_baseline=ops1['win_baseline'],
            sig_baseline=ops1['sig_baseline'],
            fs=ops1['fs'],
            prctile_baseline=ops1['prctile_baseline']
           )
# Identify spikes
       spks = suite2p.extraction.oasis(F=dF, batch_size=ops1['batch_size'], tau=ops['tau'], fs=ops['fs'])
       #ops=ops1;
       stat=stat
       np.save(os.path.join(fpath, "stat.npy"), stat)
       np.save(os.path.join(fpath, "F.npy"), F)
       np.save(os.path.join(fpath, "Fneu.npy"), Fneu)
       np.save(os.path.join(fpath, "iscell.npy"), iscell)
       np.save(os.path.join(fpath, "ops.npy"), ops)
       np.save(os.path.join(fpath, "spks.npy"), spks)

       ops_matlab=ops;
       ops_matlab['save_path']=fpath;
       suite2p.io.save_mat(ops, stat, F, Fneu, spks, iscell,redcell=-1)
    else:
       print("no ROIs found, only ops.npy file saved") 

In [6]:


# Define the number of parallel processes; Noted that current binning movie in chuns of length 06, increase this parameter could decrease the number of frames within a chunks, using less memory
num_processes = 3

# Create a list of plane indices
plane_indices = range(nplanes)

# Execute the loop in parallel
Parallel(n_jobs=num_processes)(delayed(process_plane)(plane_idx) for plane_idx in plane_indices)


[None, None, None]

If there are more than 1 plane, combine all planes data into one

In [7]:
#suite2p.io.save.combined(ops1['save_path'],save=True)
#suite2p.io.combined(ops1['save_path'],save=True)
#ops1=ops0
#ops1['save_path']='C:/Users/zhangl33/Projects/Project4-SLMonline/LuSLMOnlineTest/02082024/CommandLineProcess/'
if ops0['nplanes']>1:
   result=suite2p.io.save.combined(SaveFolder,save=True)
   stat, ops, F, Fneu, spks, iscell_0, iscell_1, redcell_0, redcell_1, hasred = result
   iscell = np.stack((iscell_0, iscell_1), axis=-1)
   #print(iscell)
   fpath=os.path.join(SaveFolder, 'combined');
if not os.path.exists(fpath):
   os.makedirs(fpath)
else:
   print("Folder already exists.")

np.save(os.path.join(fpath, "F.npy"), F)
np.save(os.path.join(fpath, "Fneu.npy"), Fneu)
np.save(os.path.join(fpath, "spks.npy"), spks)
np.save(os.path.join(fpath, "ops.npy"), ops)
np.save(os.path.join(fpath, "stat.npy"), stat)
np.save(os.path.join(fpath, "iscell.npy"), iscell)
ops_matlab=ops;
ops_matlab['save_path']=fpath;
suite2p.io.save_mat(ops, stat, F, Fneu, spks, iscell,redcell=-1)

  

appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view
Folder already exists.


Run Suite2p gui from command line, load combined data above, do necessary mannual correction. Saved the data.

Once above mannual correction is done; update the result in each plane

In [8]:
CombinePath=os.path.join(SaveFolder, 'combined');
iscellCombined = np.load(os.path.join(CombinePath, "iscell.npy"))
statCombined = np.load(os.path.join(CombinePath, "stat.npy"),allow_pickle=True)
opsCombined = np.load(os.path.join(CombinePath, "ops.npy"),allow_pickle=True)
print(len(statCombined),'of units including NON-cells in combined planes')

##extract the plane ID into a numpy variable
UnitPlane=[];
for index, Unit in enumerate(statCombined):
    UnitPlane.append(Unit['iplane'])
UnitPlane=np.array(UnitPlane)



#ops1['save_path']='C:/Users/zhangl33/Projects/Project4-SLMonline/LuSLMOnlineTest/02082024/CommandLineProcess/'
statPlane=[None]*nplanes
opsPlane=[None]*nplanes
for plane_idx in range(nplanes):
    #print(plane_idx)
    fpath=os.path.join(SaveFolder, f'plane{plane_idx}/');
    iscell=iscellCombined[UnitPlane==plane_idx,:]
    np.save(os.path.join(fpath, "iscell.npy"), iscell) ##update the iscell for each plane from the combined data
    opsPlane[plane_idx]= np.load(os.path.join(fpath, "ops.npy"),allow_pickle=True)

    ##asign information of stat for each plane to the combined planes
    statPlane[plane_idx] = np.load(os.path.join(fpath, "stat.npy"),allow_pickle=True)
    statCombined[UnitPlane==plane_idx]=statPlane[plane_idx]

stat=statCombined
#ops=opsCombined
#np.save(os.path.join(CombinePath, "ops.npy"), ops)
np.savez(os.path.join(CombinePath, "statUpdate.npz"), stat=statCombined,UnitPlane=UnitPlane)

42 of units including NON-cells in combined planes
